In [ ]:
import wikipedia
import pandas as pd

import json, glob, gc
from pandas import json_normalize

import psycopg2   
from psycopg2 import Error
import geopandas as gpd
import numpy as np

from tqdm import tqdm
from shapely.geometry import LineString, Point
from shapely.wkt import dumps, loads

from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# #first the english pages, set language to "en"
wikipedia.set_lang("pt")

#geosearch in Lisbon - 411 pages with coordinates within radius for english, 320 for portuguese
places = wikipedia.geosearch(38.736946,-9.142685, title=None, results=3000, radius=10000)

#not very efficient way, getting information about each page
geo = []
content = []
title = []
for a in places:
    try:
        b = wikipedia.WikipediaPage(title=a)
        geo.append(b.coordinates)
        content.append(b.content)
        title.append(b.title)
    except:
        continue
    
lat = []
lon = []
for a in geo:
    lat.append(float(a[0]))
    lon.append(float(a[1]))
    
#putting data into dataframe     
df = pd.DataFrame(list(zip(title, lat, lon, content)), columns=['title', 'lat', 'lon', 'content'])

#selecting those pages where the content contains the word "Alfama" = only 14
df['content'] = wikipedia_pt['content'].apply(lambda x:x.lower())
df['lang'] = 'pt'

## Filter by keyword
# df = wikipedia_pt[wikipedia_pt['content'].str.contains("alfama")]


In [ ]:
wikipedia_df_pt = wikipedia_pt.copy()
df = pd.concat([wikipedia_df_pt, wikipedia_df])

In [ ]:
table_name = ""
database = ""
user = "postgres"
password = "postgres"
port = str(5432)
host = "localhost"

engine = create_engine("postgresql://"+user+":"+password+"@"+host+":"+port+"/"+database)

def generate_geometry(x):
    return Point(x['lon'], x['lat'])

In [ ]:
df['geometry'] = df.apply(generate_geometry, axis=1)
df = df[df['geometry'].notna()]

gpd_wiki = gpd.GeoDataFrame(df, geometry='geometry')
gpd_wiki.set_crs(epsg=4326, inplace=True, allow_override=True)
gpd_wiki = gpd_wiki.__round__(10)

gpd.GeoDataFrame.to_postgis(gpd_wiki.drop(['lat','lon'], axis=1), name = table_name, con = engine, schema = 'public', if_exists = 'append')

In [ ]:
## Create table in postgis and insert data


In [ ]:
gpd_wiki[gpd_wiki['content'].str.contains("alfama")]